# Introduction: Manual Feature Engineering (part two)

In this notebook we will expand on the Introduction to Manual Feature Engineering notebook. We will use the aggregation and value counting functions developed in that notebook in order to incorporate information from the previous_application, POS_CASH_balance, installments_payments, and credit_card_balance data files. We already used the information from the bureau and bureau_balance in the previous notebook and were able to improve our competition score compared to using only the application data. After running a model with the features included here, performance does increase, but we run into issues with an explosion in the number of features! I'm working on a notebook of feature selection, but for this notebook we will continue building up a rich set of data for our model.

The definitions of the four additional data files are:

previous_application (called previous): previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV.

POS_CASH_BALANCE (called cash): monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.

credit_card_balance (called credit): monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.

installments_payment (called installments): payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment.

# Functions
We spent quite a bit of time developing two functions in the previous notebook:

agg_numeric: calculate aggregation statistics (mean, count, max, min) for numeric variables.

count_categorical: compute counts and normalized counts of each category in a categorical variable.

Together, these two functions can extract information about both the numeric and categorical data in a dataframe. Our general approach will be to apply both of these functions to the dataframes, grouping by the client id, SK_ID_CURR. For the POS_CASH_balance, credit_card_balance, and installment_payments, we can first group by the SK_ID_PREV, the unique id for the previous loan. Then we will group the resulting dataframe by the SK_ID_CURR to calculate the aggregation statistics for each client across all of their previous loans. If that's a little confusing, I'd suggest heading back to the first feature engineering notebook.**

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

# Memory management
import gc 

# 1.0.0 Function to Aggregate Numeric Data

This groups by

In [2]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    
    # First calculate counts
    counts = pd.DataFrame(df.groupby(group_var, as_index = False)[df.columns[1]].count()).rename(columns = {df.columns[1]: '%s_counts' % df_name})
    
    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg(['mean', 'max', 'min', 'sum']).reset_index()
    
    # Need to create new column names
    columns = [group_var]
    
    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
              
    #  Rename the columns
    agg.columns = columns
    
    # Merge with the counts
    agg = agg.merge(counts, on = group_var, how = 'left')
    
    return agg

# 1.0.1 Function to Calculate Categorical Counts

In [3]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

# 1.0.2 Function for KDE Plots of Variable
We also made a function that plots the distribution of variable colored by the value of TARGET (either 1 for did not repay the loan or 0 for did repay the loan). We can use this function to visually examine any new variables we create. This also calculates the correlation cofficient of the variable with the target which can be used as an approximation of whether or not the created variable will be useful.

In [4]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

Let's deal with one dataframe at a time. First up is the previous_applications. This has one row for every previous loan a client had at Home Credit. A client can have multiple previous loans which is why we need to aggregate statistics for each client.

# 1.1.0 previous_application

In [5]:
previous = pd.read_csv('previous_application.csv')
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Calculate aggregate statistics for each numeric column
previous_agg = agg_numeric(previous.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_agg.head()

,SK_ID_CURR,previous_loans_AMT_ANNUITY_mean,previous_loans_AMT_ANNUITY_max,previous_loans_AMT_ANNUITY_min,previous_loans_AMT_ANNUITY_sum,previous_loans_AMT_APPLICATION_mean,previous_loans_AMT_APPLICATION_max,previous_loans_AMT_APPLICATION_min,previous_loans_AMT_APPLICATION_sum,previous_loans_AMT_CREDIT_mean,...,previous_loans_DAYS_LAST_DUE_sum,previous_loans_DAYS_TERMINATION_mean,previous_loans_DAYS_TERMINATION_max,previous_loans_DAYS_TERMINATION_min,previous_loans_DAYS_TERMINATION_sum,previous_loans_NFLAG_INSURED_ON_APPROVAL_mean,previous_loans_NFLAG_INSURED_ON_APPROVAL_max,previous_loans_NFLAG_INSURED_ON_APPROVAL_min,previous_loans_NFLAG_INSURED_ON_APPROVAL_sum,previous_loans_counts
0,100001,3951.000,3951.000,3951.000,3951.000,24835.50,24835.5,24835.5,24835.5,23787.00,...,-1619.0,-1612.000000,-1612.0,-1612.0,-1612.0,0.000000,0.0,0.0,0.0,1
1,100002,9251.775,9251.775,9251.775,9251.775,179055.00,179055.0,179055.0,179055.0,179055.00,...,-25.0,-17.000000,-17.0,-17.0,-17.0,0.000000,0.0,0.0,0.0,1
2,100003,56553.990,98356.995,6737.310,169661.970,435436.50,900000.0,68809.5,1306309.5,484191.00,...,-3163.0,-1047.333333,-527.0,-1976.0,-3142.0,0.666667,1.0,0.0,2.0,3
3,100004,5357.250,5357.250,5357.250,5357.250,24282.00,24282.0,24282.0,24282.0,20106.00,...,-724.0,-714.000000,-714.0,-714.0,-714.0,0.000000,0.0,0.0,0.0,1
4,100005,4813.200,4813.200,4813.200,4813.200,22308.75,44617.5,0.0,44617.5,20076.75,...,-466.0,-460.000000,-460.0,-460.0,-460.0,0.000000,0.0,0.0,0.0,2


In [7]:
# Calculate value counts for each categorical column
previous_counts = count_categorical(previous, group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_counts.head()

,previous_loans_NAME_CONTRACT_TYPE_Cash loans_count,previous_loans_NAME_CONTRACT_TYPE_Cash loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_Consumer loans_count,previous_loans_NAME_CONTRACT_TYPE_Consumer loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count,previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_XNA_count,previous_loans_NAME_CONTRACT_TYPE_XNA_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm,...,previous_loans_PRODUCT_COMBINATION_POS industry without interest_count,previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm,previous_loans_PRODUCT_COMBINATION_POS mobile with interest_count,previous_loans_PRODUCT_COMBINATION_POS mobile with interest_count_norm,previous_loans_PRODUCT_COMBINATION_POS mobile without interest_count,previous_loans_PRODUCT_COMBINATION_POS mobile without interest_count_norm,previous_loans_PRODUCT_COMBINATION_POS other with interest_count,previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm,previous_loans_PRODUCT_COMBINATION_POS others without interest_count,previous_loans_PRODUCT_COMBINATION_POS others without interest_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,0.000000,1,1.000000,0,0.0,0,0.0,1,1.000000,...,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0
100002,0,0.000000,1,1.000000,0,0.0,0,0.0,0,0.000000,...,0,0.0,0,0.0,0,0.0,1,1.0,0,0.0
100003,1,0.333333,2,0.666667,0,0.0,0,0.0,1,0.333333,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,1,1.000000,0,0.0,0,0.0,1,1.000000,...,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0
100005,1,0.500000,1,0.500000,0,0.0,0,0.0,1,0.500000,...,0,0.0,1,0.5,0,0.0,0,0.0,0,0.0


In [8]:
print('Previous aggregated shape: ', previous_agg.shape)
print('Previous categorical counts shape: ', previous_counts.shape)

Previous aggregated shape:  (338857, 78)
Previous categorical counts shape:  (338857, 286)


In [9]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
previous_counts.reset_index(inplace=True)
# Merge in the previous information
train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

# Remove variables to free memory
gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

725

We are going to have to be careful about calculating too many features. We don't want to overwhelm the model with too many irrelevant features or features with too many missing values. In the previous notebook, we removed any features with more than 90% missing values. To be consistent, we will apply that same logic here.

# 1.1.1 Function to Calculate Missing Values

In [10]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df, print_info = False):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        if print_info:
            # Print some summary information
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [11]:
def remove_missing_columns(train, test, threshold = 90):
    # Calculate missing stats for train and test (remember to calculate a percent!)
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    test_miss = pd.DataFrame(test.isnull().sum())
    test_miss['percent'] = 100 * test_miss[0] / len(test)
    
    # list of missing columns for train and test
    missing_train_columns = list(train_miss.index[train_miss['percent'] > threshold])
    missing_test_columns = list(test_miss.index[test_miss['percent'] > threshold])
    
    # Combine the two lists together
    missing_columns = list(set(missing_train_columns + missing_test_columns))
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    train = train.drop(columns = missing_columns)
    test = test.drop(columns = missing_columns)
    
    return train, test

In [12]:
train, test = remove_missing_columns(train, test)

There are 6 columns with greater than 90% missing values.


# 1.2.0 Applying to More Data

Function to Aggregate Stats at the Client Level

In [13]:
def aggregate_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    
    # Aggregate the numeric columns
    df_agg = agg_numeric(df, group_var = group_vars[0], df_name = df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'object'):
    
        # Count the categorical columns
        df_counts = count_categorical(df, group_var = group_vars[0], df_name = df_names[0])
        df_counts.reset_index(inplace=True)
        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_var = group_vars[1], df_name = df_names[1])

        
    # No categorical variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_var = group_vars[1], df_name = df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

# 1.2.1 Monthly Cash Data

In [14]:
cash = pd.read_csv('POS_CASH_balance.csv')
cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [15]:
cash_by_client = aggregate_client(cash, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_by_client.head()

,SK_ID_CURR,client_cash_NAME_CONTRACT_STATUS_Active_count_mean,client_cash_NAME_CONTRACT_STATUS_Active_count_max,client_cash_NAME_CONTRACT_STATUS_Active_count_min,client_cash_NAME_CONTRACT_STATUS_Active_count_sum,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_min,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_sum,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_mean,...,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum,client_cash_counts_mean,client_cash_counts_max,client_cash_counts_min,client_cash_counts_sum,client_counts
0,100001,3.555556,4,3,32.0,0.777778,0.800000,0.750000,7.0,0.0,...,0,3.111111,7,0,28,4.555556,5,4,41,9
1,100002,19.000000,19,19,361.0,1.000000,1.000000,1.000000,19.0,0.0,...,0,0.000000,0,0,0,19.000000,19,19,361,19
2,100003,9.142857,12,7,256.0,0.928571,1.000000,0.875000,26.0,0.0,...,0,0.000000,0,0,0,9.714286,12,8,272,28
3,100004,3.000000,3,3,12.0,0.750000,0.750000,0.750000,3.0,0.0,...,0,0.000000,0,0,0,4.000000,4,4,16,4
4,100005,9.000000,9,9,99.0,0.818182,0.818182,0.818182,9.0,0.0,...,0,0.000000,0,0,0,11.000000,11,11,121,11


In [16]:
print('Cash by Client Shape: ', cash_by_client.shape)
train = train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del cash, cash_by_client
gc.collect()

Cash by Client Shape:  (337252, 174)


72

In [17]:
train, test = remove_missing_columns(train, test)

There are 0 columns with greater than 90% missing values.


# 1.2.2 Monthly Credit Data

In [19]:
credit = pd.read_csv('credit_card_balance.csv')
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [20]:
credit_by_client = aggregate_client(credit, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_by_client.head()

,SK_ID_CURR,client_credit_NAME_CONTRACT_STATUS_Active_count_mean,client_credit_NAME_CONTRACT_STATUS_Active_count_max,client_credit_NAME_CONTRACT_STATUS_Active_count_min,client_credit_NAME_CONTRACT_STATUS_Active_count_sum,client_credit_NAME_CONTRACT_STATUS_Active_count_norm_mean,client_credit_NAME_CONTRACT_STATUS_Active_count_norm_max,client_credit_NAME_CONTRACT_STATUS_Active_count_norm_min,client_credit_NAME_CONTRACT_STATUS_Active_count_norm_sum,client_credit_NAME_CONTRACT_STATUS_Approved_count_mean,...,client_credit_SK_DPD_DEF_min_sum,client_credit_SK_DPD_DEF_sum_mean,client_credit_SK_DPD_DEF_sum_max,client_credit_SK_DPD_DEF_sum_min,client_credit_SK_DPD_DEF_sum_sum,client_credit_counts_mean,client_credit_counts_max,client_credit_counts_min,client_credit_counts_sum,client_counts
0,100006,6.0,6,6,36.0,1.000000,1.000000,1.000000,6.0,0,...,0,0.0,0,0,0,6.0,6,6,36,6
1,100011,74.0,74,74,5476.0,1.000000,1.000000,1.000000,74.0,0,...,0,0.0,0,0,0,74.0,74,74,5476,74
2,100013,96.0,96,96,9216.0,1.000000,1.000000,1.000000,96.0,0,...,0,1.0,1,1,96,96.0,96,96,9216,96
3,100021,7.0,7,7,119.0,0.411765,0.411765,0.411765,7.0,0,...,0,0.0,0,0,0,17.0,17,17,289,17
4,100023,8.0,8,8,64.0,1.000000,1.000000,1.000000,8.0,0,...,0,0.0,0,0,0,8.0,8,8,64,8


In [21]:
print('Credit by client shape: ', credit_by_client.shape)

train = train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del credit, credit_by_client
gc.collect()

Credit by client shape:  (103558, 398)


71

In [22]:
train, test = remove_missing_columns(train, test)

There are 0 columns with greater than 90% missing values.


# 1.2.3 Installment Payments

In [23]:
installments = pd.read_csv('installments_payments.csv')
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [24]:
installments_by_client = aggregate_client(installments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
installments_by_client.head()

,SK_ID_CURR,client_installments_SK_ID_CURR_mean_mean,client_installments_SK_ID_CURR_mean_max,client_installments_SK_ID_CURR_mean_min,client_installments_SK_ID_CURR_mean_sum,client_installments_SK_ID_CURR_max_mean,client_installments_SK_ID_CURR_max_max,client_installments_SK_ID_CURR_max_min,client_installments_SK_ID_CURR_max_sum,client_installments_SK_ID_CURR_min_mean,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_PAYMENT_sum_mean,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_PAYMENT_sum_min,client_installments_AMT_PAYMENT_sum_sum,client_installments_counts_mean,client_installments_counts_max,client_installments_counts_min,client_installments_counts_sum,client_counts
0,100001,100001,100001,100001,700007,100001,100001,100001,700007,100001,...,27746.775,21834.096429,29250.900,11945.025,1.528387e+05,3.571429,4,3,25,7
1,100002,100002,100002,100002,1900038,100002,100002,100002,1900038,100002,...,175783.725,219625.695000,219625.695,219625.695,4.172888e+06,19.000000,19,19,361,19
2,100003,100003,100003,100003,2500075,100003,100003,100003,2500075,100003,...,1154108.295,453952.220400,1150977.330,80773.380,1.134881e+07,9.160000,12,6,229,25
3,100004,100004,100004,100004,300012,100004,100004,100004,300012,100004,...,16071.750,21288.465000,21288.465,21288.465,6.386540e+04,3.000000,3,3,9,3
4,100005,100005,100005,100005,900045,100005,100005,100005,900045,100005,...,43318.800,56161.845000,56161.845,56161.845,5.054566e+05,9.000000,9,9,81,9


In [25]:
print('Installments by client shape: ', installments_by_client.shape)

train = train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del installments, installments_by_client
gc.collect()

Installments by client shape:  (339587, 118)


73

In [26]:
train, test = remove_missing_columns(train, test)

There are 0 columns with greater than 90% missing values.


In [27]:
print('Final Training Shape: ', train.shape)
print('Final Testing Shape: ', test.shape)

Final Training Shape:  (307511, 1166)
Final Testing Shape:  (48744, 1165)


# 3.0.0 Save All Newly Calculated Features
Unfortunately, saving all the created features does not work in a Kaggle notebook. You will have to run the code on your personal machine. I have run the code and uploaded the entire datasets here. I plan on doing some feature selection and uploading reduced versions of the datasets. Right now, they are slightly to big to handle in Kaggle notebooks or scripts. .

In [28]:
train.to_csv('train_previous_raw.csv', index = False, chunksize = 500)
test.to_csv('test_previous_raw.csv', index = False)